In [1]:
import cv2, os
import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow, figure
rng = np.random.RandomState(1234)
import theano
#%matplotlib inline
np.set_printoptions(threshold=np.inf)

In [2]:
cascadePath = "/usr/share/opencv/haarcascades/haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);

In [3]:
import glob, os
normalised_image_array=np.empty([150, 1024])
lst=os.listdir("/home/manchanda/YALEFACS/Images/")
lst.sort()
i=0
for file in lst:
    image_pil = Image.open("/home/manchanda/YALEFACS/Images/"+file).convert('L')
    image = np.array(image_pil, 'uint8')
    #figure()
    #imshow(np.asarray(image))
    faces = faceCascade.detectMultiScale(
        image,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(32, 32),
        #flags = cv2.CV_HAAR_SCALE_IMAGE
        #flags = cv2.CASCADE_SCALE_IMAGE
        flags = 0
    )
    face_img=0
    for (x, y, w, h) in faces:
        face_img=image[y: y + h, x: x + w]
    #figure()
    #imshow(np.asarray(face_img))
    resized_image = cv2.resize(face_img, (32, 32))
    #figure()
    #imshow(np.asarray(resized_image))
    #print resized_image
    resized_image = resized_image.reshape(1,1024)
    
    #
    #NORMALISE
    #
    #
    eps=1e-8
    resized_image = resized_image.copy()
    resized_image -= resized_image.min()
    divide = (resized_image.max() + eps)
    #print divide
    resized_image = resized_image / divide
    normalised_image_array[i]=resized_image
    i=i+1

In [4]:
normalised_image_array.shape
normalised_image_array_without_shuffle = normalised_image_array;
#np.random.shuffle(normalised_image_array)

In [5]:
n_in=16
n_out=1024
W_values = np.asarray(
                rng.uniform(
                    low=-np.sqrt(6. / (n_in + n_out)),
                    high=np.sqrt(6. / (n_in + n_out)),
                    size=(n_in, n_out)
                ),
                dtype=theano.config.floatX
            )
print W_values
print W_values.shape
print type(W_values)

[[ -4.68615595e-02   1.85496539e-02  -9.45983553e-03   4.33490807e-02
    4.25313784e-02  -3.45456631e-02  -3.39575173e-02   4.58576757e-02
    6.95963639e-02   5.71082668e-02  -2.15991072e-02   1.51170419e-04
    2.78700205e-02   3.23117574e-02  -1.97103253e-02   9.29636801e-03
    4.68366432e-04  -7.38638749e-02   4.14453390e-02   5.81273696e-02
   -2.05252925e-02   1.75299379e-02  -6.45042199e-02  -1.99270640e-02
    6.57987049e-02   2.29959907e-02  -1.56160495e-02   4.38612573e-02
   -2.78245905e-02   1.03449280e-02   5.60744758e-02  -9.69595294e-03
    4.58995218e-02  -5.41157041e-02   3.10294690e-02   3.10781317e-02
   -4.27185457e-02   6.45420260e-02  -8.78945021e-03   6.21795578e-02
   -6.68697794e-02  -4.79602348e-02  -6.87616693e-02   2.65663224e-02
    1.43745360e-02   5.06017701e-03  -6.93740550e-02   9.33235479e-03
   -2.58752206e-02   4.50694303e-04  -5.89574855e-02   1.62838929e-02
    1.00177104e-02  -7.49279178e-02   1.78406763e-02   6.26059607e-02
    4.41337840e-02  

In [6]:
def accumulate(resized_image,W_values):
    y=np.zeros(16)
    #print W_values
    for i in range(0,16):
        y[i] = 0
        for j in range(0,1024):
            y[i]= y[i] + W_values[i][j]*resized_image[j]
    return y

In [7]:
def update(resized_image,W_values,y):
    for j in range(0,1024):
        PSij=0
        for i in range(0,16):
            delta=0
            PSij+=y[i]*W_values[i][j]
            delta = 0.0001*y[i]*(resized_image[j] - PSij)
            W_values[i][j] += delta
    return W_values

In [8]:
for i in range(0,100):
    for j in range(0,150):
        y = accumulate(normalised_image_array[j],W_values)
        #print y
        W_values = update(normalised_image_array[j],W_values,y)
    print i

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [9]:
print W_values

[[  4.41371951e-02   2.84754075e-02   1.79612422e-02   9.49078193e-03
    7.39931961e-03   7.07261199e-03   1.04176091e-02   1.40768800e-02
    1.66914574e-02   1.83514993e-02   2.01013441e-02   2.08050890e-02
    2.15051801e-02   2.21174472e-02   2.31627328e-02   2.44650099e-02
    2.58916658e-02   2.78106251e-02   2.86672638e-02   2.80140520e-02
    2.63899581e-02   2.35842102e-02   2.02473401e-02   1.77819142e-02
    1.47224991e-02   1.00809481e-02   7.36464206e-03   7.93456893e-03
    9.86168779e-03   1.27920650e-02   2.11640739e-02   3.33548080e-02
    3.97550172e-02   2.27859005e-02   1.07713743e-02   7.69739813e-03
    6.12128461e-03   1.00013404e-02   1.53080859e-02   1.81840650e-02
    2.10605880e-02   2.40035904e-02   2.67175580e-02   2.92394374e-02
    3.06323035e-02   3.12984928e-02   3.20358471e-02   3.30642136e-02
    3.41948736e-02   3.51705707e-02   3.49334498e-02   3.52134805e-02
    3.28814399e-02   2.86382607e-02   2.40788250e-02   2.05661885e-02
    1.79290022e-02  

In [10]:
yp=np.empty([150, 16])
for i in range(0,150):
    yp[i] = accumulate(normalised_image_array_without_shuffle[i],W_values)

In [11]:
print W_values.shape
print yp.shape
np.savetxt("/home/manchanda/Projects/EmbeddedSystems/Weights_after_training.csv", W_values, delimiter=",")
np.savetxt("/home/manchanda/Projects/EmbeddedSystems/yp_after_training.csv", yp, delimiter=",")

(16, 1024)
(150, 16)
